In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import dash_cytoscape as cyto

## Функции

In [2]:
def create_prot_net(dis, distable):
    dfprot1 = distable[distable.diseases == str(dis)]
    listofnodes = dfprot1['diseases'].unique().tolist() + dfprot1['gene'].values.tolist()
    
    if len(listofnodes) <=12:
        pass
    else:
        listofnodes = listofnodes[:11]
    pp = [
    { 
        'data': {'id': listofnodes[0]},
        'classes': 'pink'}
    ]
    nodes = [
    {
        'data': {'id': x, 'label': x},
        'classes': 'rectangle'
    }
    for x in listofnodes[1:]]
    edges = [
    {'data': {'source': listofnodes[0], 'target': target}}
    for target in listofnodes[1:]
    ]
    elements = pp + nodes + edges
    return elements

In [3]:
def create_dis_net(prot, distable):
    dfprot1 = distable[distable.gene == str(prot)]
    listofnodes = dfprot1['gene'].unique().tolist() + dfprot1['diseases'].values.tolist()
    
    if len(listofnodes) <=12:
        pass
    else:
        listofnodes = listofnodes[:11]
    pp = [
    {
        'data': {'id': listofnodes[0]},
        'classes': 'rectangle'}
    ]
    nodes = [
    {
        'data': {'id': x, 'label': x},
        'classes': 'pink'
    }
    for x in listofnodes[1:]]
    edges = [
    {'data': {'source': listofnodes[0], 'target': target}}
    for target in listofnodes[1:]
    ]
    elements = pp + nodes + edges
    return elements

In [4]:
def create_list_proteins(dis, distable):
    dfprot1 = distable[distable.diseases == str(dis)]
    listofproteins = dfprot1['gene'].values.tolist()
    return [{'label': categ, 'value': categ} for categ in sorted(listofproteins)]

## Загрузка данных

In [5]:
distable = pd.read_csv('human_disease_experiments_full.tsv', 
                       sep = '\t', 
                       names = ['ENSP', 'gene', 'DOID', 'diseases', 'TIGA', 'MenRankScore', 'score'])
distable = distable[distable['diseases'].str.contains('ICD10')==False]
distable = distable[distable['diseases'].str.contains('DOID:')==False]
distable = distable[distable['diseases'].eq('Disease')==False]
distable = distable.sort_values(by=['gene', 'score'], ascending=False)

In [6]:
available_dis = sorted(distable['diseases'].unique())
listofproteins = sorted(distable['gene'].unique())

## Фильтры

In [7]:
disprot_filter =dbc.Col([
        dcc.Dropdown(
            id="disprot_dropdown",
            value='Disease of anatomical entity',
            options=[{'label': category, 'value': category} for category in available_dis]
        ),
    ],
    style={'max-width': '100%'},
)

In [8]:
protdis_filter = dbc.Col([
        dcc.Dropdown(
            id="protdis_dropdown",
            value=None,
            options=[{'label': category, 'value': category} for category in listofproteins]
        ),
    ],
    style={'max-width': '100%'},
)

## Приложение

In [9]:
app = dash.Dash(external_stylesheets=["assets/html-components.css", dbc.themes.BOOTSTRAP])

## Карточки

In [10]:
disprotcard = dbc.Card(
    [
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.Label("Заболевание — Белки",
                               style={'font-size': 24,
                                      'text-align': 'left',
                                      },
                               ),
                ], width=6),
                dbc.Col([
                    disprot_filter,
                ], width=6)
            ]),
            cyto.Cytoscape(
                id='cytoscape-disprot-callbacks',
                layout={'name': 'cose'},
                stylesheet=[
                    {'selector': 'node',
                     'style': {
                         'background-color': '#00BFFF',
                         'label': 'data(label)'}
                    },
                    {'selector': 'edge',
                     'style': {
                         'line-color': '#A3C4BC'}
                    },
                    {
                    'selector': '.rectangle',
                    'style': {
                        'shape': 'rectangle'}},
                    {'selector': '.pink',
                     'style': {
                         'background-color': '#FF69B4'}}],
                style={'width': '100%', 'height': '100%'},
                )
        ],
            style={
                'height': '42rem',
            }
        )
    ])

In [11]:
prdiscard = dbc.Card(
    [
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.Label("Белок — Заболевания",
                               style={'font-size': 24,
                                      'text-align': 'left',
                                      },
                               ),
                ], width=6),
                dbc.Col([
                    protdis_filter,
                ], width=6)
            ]),
            cyto.Cytoscape(
                id='cytoscape-protdis-callbacks',
                layout={'name': 'cose'},
                stylesheet=[
                    {'selector': 'node',
                     'style': {
                         'background-color': '#00BFFF',
                         'label': 'data(label)'}
                    },
                    {'selector': 'edge',
                     'style': {
                         'line-color': '#A3C4BC'}
                    },
                    {
                    'selector': '.rectangle',
                    'style': {
                        'shape': 'rectangle'}},
                    {'selector': '.pink',
                     'style': {
                         'background-color': '#FF69B4'}}],
                style={'width': '100%', 'height': '100%'},
                )
        ],
            style={
                'height': '42rem',
            }
        )
    ])

In [12]:
app.layout = html.Div(children=[
    html.H1('Взаимосвязь белков с заболеваниями'),
    html.Br(),
    dbc.Row([
        dbc.Col([
            disprotcard,
        ]),
        dbc.Col([
            prdiscard,
        ])])
], style={'margin-left':'80px',
         'margin-right':'80px'}
)

## Колбеки

In [13]:
@app.callback(
    Output('protdis_dropdown', 'options'),
    Input('disprot_dropdown', 'value')
)
def update_protdis_dropdown(dis):
    return create_list_proteins(dis, distable)

In [14]:
@app.callback(
    Output('cytoscape-disprot-callbacks', 'elements'),
    Input('disprot_dropdown', 'value')
)
def create_prot_net_upd(prot):
    return create_prot_net(prot, distable)

In [15]:
@app.callback(
    Output('cytoscape-protdis-callbacks', 'elements'),
    Input('protdis_dropdown', 'value')
)
def create_dis_net_upd(prot):
    return create_dis_net(prot, distable)

In [16]:
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [26/Jul/2022 01:03:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 01:03:08] "GET /assets/html-components.css HTTP/1.1" 404 -
127.0.0.1 - - [26/Jul/2022 01:03:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 01:03:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 01:03:08] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Jul/2022 01:03:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 01:03:08] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Programs\Anaconda\envs\ds\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Programs\Anaconda\envs\ds\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Programs\Anaconda\envs\ds\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Programs\Anaconda\envs\ds\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Programs\Anaconda\envs\ds\lib\site-packages\dash\dash.py", line 1472, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Programs\Anaconda\envs\ds\lib\site-packages\dash\_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callba

127.0.0.1 - - [26/Jul/2022 01:03:08] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/Jul/2022 01:03:15] "POST /_dash-update-component HTTP/1.1" 200 -
